<a href="https://colab.research.google.com/github/fastai-energetic-engineering/ashrae/blob/master/kaggle_data_to_parquet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Kaggle Data for ASHRAE Energy Prediction
> "How to download Kaggle data from Colab."

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [kaggle, preprocessing]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: false


In [ ]:
#collapse
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 7.2MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 1.2MB 33.7MB/s 
     |████████████████████████████████| 194kB 51.3MB/s 
     |████████████████████████████████| 61kB 10.5MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
Mounted at /content/gdrive


In [ ]:
#collapse
from fastbook import *
import os
from google.colab import files
import pandas as pd
import datetime

This notebook demonstrates how I downloaded the [ASHRAE Energy Prediction Data](https://www.kaggle.com/c/ashrae-energy-prediction/overview) from Kaggle.

First, we need to install the [Kaggle API](https://github.com/Kaggle/kaggle-api#api-credentials).

In [ ]:
!pip install kaggle --upgrade -q

I will download the data into a folder in my google drive. First, I will set my home directory.

In [ ]:
%cd content/gdrive/MyDrive/Colab Notebooks/ashrae/

[Errno 2] No such file or directory: 'content/gdrive/MyDrive/Colab Notebooks/ashrae/'
/content


We need to download Kaggle API token and then put the `.json` file in `.kaggle` folder. We can upload the key directly from colab.

In [ ]:
files.upload() # use this to upload your API json key
!mkdir ~/.kaggle # create folder
!cp kaggle.json ~/.kaggle/ # move the key into the folder
!chmod 600 ~/.kaggle/kaggle.json # change permissions of the file

Saving kaggle.json to kaggle.json


We can finally download the file!

In [ ]:
os.mkdir('data') # make a data folder
os.chdir('data') # move to data folder
!kaggle competitions download -c ashrae-energy-prediction

  0% 0.00/1.27M [00:00<?, ?B/s]
100% 1.27M/1.27M [00:00<00:00, 89.1MB/s]
  0% 0.00/2.53M [00:00<?, ?B/s]
100% 2.53M/2.53M [00:00<00:00, 172MB/s]
 74% 65.0M/88.4M [00:00<00:00, 79.6MB/s]
100% 88.4M/88.4M [00:00<00:00, 121MB/s] 
  0% 0.00/44.5k [00:00<?, ?B/s]
100% 44.5k/44.5k [00:00<00:00, 46.4MB/s]
100% 166M/167M [00:01<00:00, 107MB/s] 
100% 167M/167M [00:01<00:00, 112MB/s]
 88% 105M/120M [00:01<00:00, 66.5MB/s] 
100% 120M/120M [00:01<00:00, 88.2MB/s]


In [ ]:
# extract zip files then remove the .zip
for item in os.listdir(): # for every item in the folder
    if item.endswith('.zip'): # check if it is a .zip file
        file_extract(item) # if it is, then extract file
        os.remove(item) # and then remove the .zip

In [ ]:
os.chdir("..") # return to initial folder

In [ ]:
def prepare_data(dataset="train"):
    assert dataset in ["train", "test"]

    # read data
    building_df = pd.read_csv("data/building_metadata.csv")
    weather_df = pd.read_csv(f"data/weather_{dataset}.csv")
    data_df = pd.read_csv(f"data/{dataset}.csv")

    # convert datetime
    data_df["timestamp"] = pd.to_datetime(data_df["timestamp"])
    data_df = data_df.pivot(
        index=["timestamp", "building_id"], columns="meter", values="meter_reading"
    )
    data_df.columns = [f"meter_{meter_type}" for meter_type in data_df.columns]
    data_df = data_df.fillna(0.0).reset_index()

    # adjust timestamp
    timediff = {
        0: 4,
        1: 0,
        2: 7,
        3: 4,
        4: 7,
        5: 0,
        6: 4,
        7: 4,
        8: 4,
        9: 5,
        10: 7,
        11: 4,
        12: 0,
        13: 5,
        14: 4,
        15: 4,
    }
    weather_df["time_diff"] = weather_df["site_id"].map(timediff)
    weather_df["time_diff"] = weather_df["time_diff"].apply(
        lambda x: datetime.timedelta(hours=x)
    )
    weather_df["timestamp_gmt"] = pd.to_datetime(weather_df["timestamp"])
    weather_df["timestamp"] = weather_df["timestamp_gmt"] - weather_df["time_diff"]

    # merge table
    data_df = data_df.merge(building_df, on="building_id", how="left")
    data_df = data_df.merge(
        weather_df.drop(columns=["time_diff", "timestamp_gmt"]),
        on=["site_id", "timestamp"],
        how="left",
    )

    return data_df

Let's try this function out!

In [ ]:
train_combined_df = prepare_data('train')

In [ ]:
train_combined_df.shape

(12393999, 18)

In [ ]:
train_combined_df.to_parquet(Path("/content/gdrive/MyDrive/Colab Notebooks/ashrae/train_combined.snappy.parquet"))